In [1]:
import os
import boto3
import numpy as np
import tensorflow as tf
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name


In [2]:
training_data_uri = 's3://sagemaker-us-east-1-731833107751/mnist-data'
print(training_data_uri)

s3://sagemaker-us-east-1-731833107751/mnist-data


In [ ]:
# Store in S3

# s3 = boto3.resource("s3")
# for bucket in s3.buckets.all():
#     print(bucket.name)
    

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

np.save('/tmp/x_train', x_train)
np.save('/tmp/y_train', y_train)
np.save('/tmp/x_test', x_test)
np.save('/tmp/y_test', y_test)
#s3.Bucket('sagemaker-us-east-1-731833107751').upload_file('ec2_file', 's3_file')

In [ ]:
# s3_path = 's3://' + sagemaker_session.default_bucket() + '/'
# mnist_data_path = s3_path + "/mnist_data"
# print (s3_path)

In [4]:
prefix = 'mnist-data'

training_input_path = sagemaker_session.upload_data('/tmp/x_train.npy', key_prefix=prefix)
training_label_path = sagemaker_session.upload_data('/tmp/y_train.npy', key_prefix=prefix)
test_input_path = sagemaker_session.upload_data('/tmp/x_test.npy', key_prefix=prefix)
test_label_path = sagemaker_session.upload_data('/tmp/y_test.npy', key_prefix=prefix)

In [ ]:
# s3.Bucket(s3_path).upload_file('/tmp/x_train.npy','mnist-data/x_train.npy') 
# s3.Bucket(s3_path).upload_file('/tmp/y_train.npy', 'mnist-data/y_train.npy')
# s3.Bucket(s3_path).upload_file('/tmp/x_test.npy', 'mnist-data/x_test.npy')
# s3.Bucket(s3_path).upload_file('/tmp/y_test.npy', 'mnist-data/y_test.npy')

In [ ]:
# TensorFlow 2.0 script
!pygmentize 'mnist-2.py'

In [5]:
from sagemaker.tensorflow import TensorFlow
mnist_estimator2 = TensorFlow(entry_point='mnist-2.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='local',
                             #train_instance_type='ml.m5.4xlarge',
                             framework_version='2.0.0',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': True}})

In [6]:
mnist_estimator2.fit(training_data_uri)

Creating tmp29bpqx7y_algo-1-kakhd_1 ... 
Attaching to tmp29bpqx7y_algo-1-kakhd_12mdone
algo-1-kakhd_1  | 2020-02-05 23:25:45,819 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-kakhd_1  | 2020-02-05 23:25:45,827 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-kakhd_1  | 2020-02-05 23:25:46,168 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-kakhd_1  | 2020-02-05 23:25:46,191 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-kakhd_1  | 2020-02-05 23:25:46,210 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-kakhd_1  | 2020-02-05 23:25:46,223 sagemaker-containers INFO     Invoking user script
algo-1-kakhd_1  | 
algo-1-kakhd_1  | Training Env:
algo-1-kakhd_1  | 
algo-1-kakhd_1  | {
algo-1-kakhd_1  |     "additional_framework_parameters": {
algo-1-kakhd_1  |         "sagemaker_parameter_server_enabled": 

In [ ]:
predictor2 = mnist_estimator2.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

In [ ]:
# # Save the model to a temporary directory
# import tempfile

# MODEL_DIR = tempfile.gettempdir()
# version = 1
# export_path = os.path.join(MODEL_DIR, str(version))

# print('export_path = {}\n'.format(export_path))
# if os.path.isdir(export_path):
#     print('\nAlready saved a model, cleaning up\n')
#     !rm -r {export_path}
    
# tf.saved_model.save(model, export_path)
# # model.save(export_path)

# print('\nSaved model:')
# !ls -l {export_path}

In [ ]:
#tf.saved_model.contains_saved_model(export_path)


In [ ]:
#!saved_model_cli show --dir {export_path} --all


In [ ]:
#new_model = tf.saved_model.load(export_path)

In [ ]:
#outputs = new_model(x_test[1])

In [ ]:
#f = new_model.signatures["serving_default"]
#print(f(x=tf.constant([[1.]])))